In [0]:
#2.autograd
#对于python神经网络来说，autograd是一个非常关键的库，主要是提供对tensors上所有运算操作的自动微分，计算梯度功能
#它属于define-by-run类型框架，即反向传播操作的定义是根据代码的运行方式，因此每次的迭代都可以是不同的

#2.1张量
'''
  torch.Tensor是Pytorch中最主要的库，当设置其属性为.requires_grad = True, 那么就会开始追踪在该变量上的所有操作，而完成计算后，可以调用.backward()并自动计算所有梯度，得到的梯度都保存在属性.grad()中
  上边这句话说得实在太抽象了，还要在实战中理解，总的来看是一个自动计算梯度用到的东西？

  调用.detach()方法分离出计算的历史，可以停止一个tensor变量继续追踪其历史信息，同时也防止未来计算时候被追踪

  …… 看理论看不懂，还实先从代码开始研究

'''
import torch

x = torch.ones(2,2,requires_grad = True)#need gradient
print(x)

y = x + 2
print(y)# 重点！ y是一个操作的结果，所以它带有属性grad_fn
print(y.grad_fn)

z = y * y * 3
out = z.mean()
print("z = ",z)
print("out = ",out)

#实际上，一个Tensor的变量默认requires_grad 是False， 可以像上边一样在定义时候指定属性为true，也可以在定义变量后 调用a.requires_grad_(True) 这里的_代表了会对原有的属性进行修改



In [12]:
a = torch.randn(2,2)
a = ((a*3)/(a-1))
print(a.requires_grad)
a.requires_grad = True
print(a.requires_grad)
b = (a * a).sum()
print(b.grad_fn)

False
True


In [13]:
#2.2 梯度，接下来就是计算梯度，进行反向传播操作。
#上节所述，out变量定义是一个标量，因此out.backward() 相当于 out.backward(torch.tensor(1.)) 
#这句话也太难理解了吧

out.backward()
#输出梯度 d(out)/dx
print(x.grad)

#梯度公式还是不太熟练，但是这里能推导出4.5是因为这个初值是全1，也就是x = 1这类的

tensor([[4.5000, 4.5000],
        [4.5000, 4.5000]])


In [19]:
#如果从数学上来说，有一个向量值函数，那么对应的梯度应该是一个jacobian matrix
#一般来说，torch.autograd 就是用于计算雅克比向量乘积的工具，直接进入代码练习环节

x = torch.randn(3,requires_grad=True) #[a,b,c]

y  = x * 2
while y.data.norm() < 1000: #直接说L2范数就懂了，距离也就是绝对长度
  y = y * 2

print(y)
#y不是一个标量了，也不能直接计算完整的jabian行列式，但我们可以通过简单传递给向量backward()方法作为jacobian响亮的乘积，例子如下
v = torch.tensor([0.1,1.0,0.0001],dtype = torch.float)
y.backward(v)
print(x.grad) #指的是相对于x的梯度？


print(x.requires_grad)
print((x**2).requires_grad)

with torch.no_grad():#停止追踪变量历史进行自动梯度计算 这个还是很玄学，梯度计算应该是用在反向传播中的， 但就是怎么也理解不了这回事啊...
  print((x**2).requires_grad)

tensor([-498.7531,  666.8201,  792.0768], grad_fn=<MulBackward0>)
tensor([5.1200e+01, 5.1200e+02, 5.1200e-02])
True
True
False
